# import 

In [8]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
import itertools
from PIL import ImageFont, ImageDraw, Image
import pyttsx3
import threading
#pip install playsound==1.2.2

# function

In [3]:
#각도 계산 함수
def calculateAngle(landmark1, landmark2, landmark3): 
    # Get the required landmarks coordinates.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # Check if the angle is less than zero.
    if angle < 0:

        # Add 360 to the found angle.
        angle += 360
    
    # Return the calculated angle.
    return angle

In [4]:
#이미지 파일을 mediapipe에 적용하는 함수
def detectPose(image, pose, display=True):

    # Create a copy of the input image.
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform the Pose Detection.
    results = pose.process(imageRGB)
    
    # Retrieve the height and width of the input image.
    height, width, _ = image.shape
    
    # Initialize a list to store the detected landmarks.
    landmarks = []
    
    # Check if any landmarks are detected.
    if results.pose_landmarks:
    
        # Draw Pose landmarks on the output image.
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)
        
        # Iterate over the detected landmarks.
        for landmark in results.pose_landmarks.landmark:
            
            # Append the landmark into the list.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                                  (landmark.z * width)))
    
    # Check if the original input image and the resultant image are specified to be displayed.
    if display:
    
        # Display the original input image and the resultant image.
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
        # Also Plot the Pose landmarks in 3D.
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
        
    # Otherwise
    else:
        
        # Return the output image and the found landmarks.
        return output_image, landmarks

In [5]:
#특정 동작 조건 설정 함수
def squat_classifyPose(landmarks, output_image, display=False):
    count=[]
    label = 'Unknown Pose'
    color = (0, 0, 255)
    
    left_knee_angle,left_waist_angle=squat_anglecheck(landmarks)
    
    if 290 < left_knee_angle < 310:
        if 45 < left_waist_angle <65:
            label = 'sit Pose' 
   
    if 180 <left_knee_angle < 270:# or right_knee_angle > 160 and right_knee_angle < 195:
        if 130  <left_waist_angle< 190:         
                label = 'stand Pose'       
         
        

    if label != 'Unknown Pose':
        color = (0, 255, 0)  
    
    cv2.putText(output_image, label, (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
    if display:

        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
    else:
        return output_image, label

In [6]:
#무릎, 허리 각도 추출 함수
def squat_anglecheck(landmark):
    left_knee_angle = calculateAngle(landmark[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmark[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmark[mp_pose.PoseLandmark.LEFT_ANKLE.value])
    
    left_waist_angle = calculateAngle(landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                     landmark[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmark[mp_pose.PoseLandmark.LEFT_KNEE.value])
    return left_knee_angle,left_waist_angle

In [7]:
#최근 4개의 값을 비교하여 무릎이 구부러지는 상태를 분류
def down_up(stack_angless):
    down_up=0
    #구부러지는 중
    if len(stack_angless)>3:
            i= len(stack_angless)-1
            if (stack_angless[i] - stack_angless[i-1] >= 1) and \
   (stack_angless[i-1] - stack_angless[i-2] >= 1) and \
   (stack_angless[i-2] - stack_angless[i-3] >= 1):
                down_up='-'
    #펴지는 중
            if (stack_angless[i-1] - stack_angless[i] >= 1) and \
   (stack_angless[i-2] - stack_angless[i-1] >= 1) and \
   (stack_angless[i-3] - stack_angless[i-2] >= 1) :
                down_up='+'
            else :
                dwon_up='x'
    return down_up

In [8]:
# +,-의 카운트를 비교하여 앉는 중, 서는 중 표현
def in_action(stack_down_up):
    if len(stack_down_up)>4:
        last_five = stack_down_up[-5:] 
        plus_count = last_five.count('+') 
        minus_count = last_five.count('-')
        #서는 중
        if plus_count > minus_count:
            return 'standing'
        #앉는 중
        elif plus_count < minus_count:
            return 'sitting'
        #중립
        else:
            return 'neutrality'    

In [9]:
#서는 동작->앉는 동작
def feedback_timing(stack_in_action):
    label=0
    index=len(stack_in_action)-20
    new_stack_in_action=stack_in_action[-20:]
    check_stack_angless=ar[index:]
    new_stack_in_action1=new_stack_in_action.copy()
    while 'neutrality'  in new_stack_in_action1:
        new_stack_in_action1.remove('neutrality') 
            
    if len(stack_in_action)>=20:
        #rint(new_stack_in_action)
        if new_stack_in_action.count('neutrality')>=10 and new_stack_in_action[-3:]==['sitting']*3:
             label= sitting_start_feedback(check_stack_angless, label)  
        if new_stack_in_action.count('neutrality')>=10 and new_stack_in_action[-3:]==['standing']*3:
             label= standing_start_feedback(check_stack_angless, label)
            
            
        if 'standing' in new_stack_in_action1:
            if new_stack_in_action1[-3:]==['sitting','sitting','sitting']:
                label= sitting_start_feedback(check_stack_angless, label)
        if 'sitting' in new_stack_in_action1:
            if new_stack_in_action1[-3:]==['standing','standing','standing']:
                label= standing_start_feedback(check_stack_angless, label)
    return label

In [10]:
def sitting_start_feedback(check_stack_angless, label):
    if 160 <min(check_stack_angless[:,0])<=200:
        label = 'perfect'
    if 200 <min(check_stack_angless[:,0])< 270 :
        label = 'wake up more'
     
    return label

In [11]:
def standing_start_feedback(check_stack_angless, label):
    max_val=max(check_stack_angless[:,0])
    where_val=np.where(check_stack_angless[:,0]==max_val)
    if len(where_val)>1:
        max_index_ar=int(list(where_val)[0])
    else:
        max_index_ar=where_val
    if max_index_ar:
        max_index=int(max_index_ar[0])
    if 290 <=max(check_stack_angless[:,0])<320:
        if check_stack_angless[max_index,1]>60:
            label = 'perfect'
        else:
            label = 'straighten your back'
    if 270 <max(check_stack_angless[:,0])< 290 :
        if check_stack_angless[max_index,1]>60:
            label = 'sit down more'
        else:
            label = 'sit down more and straighten your back'
    return label

# test

In [ ]:
# 라이브러리 불러오기
import cv2
import threading
import time
import pyttsx3
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)
color = (0, 0, 255)
color1 = (128, 0, 0)
color2 = (0,255,0)
stack_label=[]
stack_angless=[]
no_stack_label=[]
count_check=[]
stack_down_up=[]
stack_in_action=[]
no_counting=0
real_count=0
stack_no_count=[]
ar=np.empty((0,2),float)
stack_label2=[]

camera_video = cv2.VideoCapture(0)
camera_video.set(cv2.CAP_PROP_FPS, 20)#프레임 설정
camera_video.set(3,1280)
camera_video.set(4,960)
event = threading.Event()
ar=np.empty((0,2),float)


def speak_label(label):
    # Speak label using pyttsx3
    engine = pyttsx3.init()
    engine.setProperty("rate", 300)
    engine.say(label)
    engine.runAndWait()

    # Set event flag to indicate speech is finished
    event.set()
    
def display_frame():
    global ar
    while camera_video.isOpened():
        new_label=[]
        count=0
        engine = pyttsx3.init()
    # Read a frame.
        ok, frame = camera_video.read()
    
    # Check if frame is not read properly.
        if not ok:
        
        # Continue to the next iteration to read the next frame and ignore the empty camera frame.
            continue
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
        frame = cv2.flip(frame, 1)

        # Get the width and height of the frame
        frame_height, frame_width, _ =  frame.shape

        # Resize the frame while keeping the aspect ratio.
        frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))

        # Perform Pose landmark detection.
        frame, landmarks = detectPose(frame, pose_video, display=False)

        # Check if the landmarks are detected.
        if landmarks:
            # Perform the Pose Classification.)
            frame, label = squat_classifyPose(landmarks, frame, display=False)

           #앉고, 서고, 카운팅 
            stack_label.append(label)
            new_label = [key for key, group in itertools.groupby(stack_label) if key != 'Unknown Pose' ]
            if len(new_label)>=2:
            
                for i in range(len(new_label)-1):
                    if new_label[i:i+2] == ['sit Pose', 'stand Pose']:
                        count += 1
           #노카운팅
            angless1, angless2=squat_anglecheck(landmarks)
            arr = np.array([[angless1,angless2]])
            ar=np.append(ar,arr,axis=0)
            ing=down_up(ar[:,0])
            stack_down_up.append(ing)
            in_actions=in_action(stack_down_up)
            stack_in_action.append(in_actions)
            label2=feedback_timing(stack_in_action)

            #시각화
            if label2!=0:
                cv2.putText(frame,str(label2), (10, 600),cv2.FONT_HERSHEY_PLAIN, 3, color, 2)
                stack_label2.append(label2)
#---------------------ㅈㅓㄴㄷㅏㄹㅇㅗㅏㄴㄹㅛ-----------#
            if len(stack_label2)>1:
                if stack_label2[-1]!=stack_label2[-2] and stack_label2[-2]==stack_label2[-3]:
                    if stack_label2[-1]!='perfect':
                        t = threading.Thread(target=speak_label, args=(stack_label2[-1],))
                        t.start()
            # Reset event flag to indicate speech is in progress
                        event.clear()

            cv2.putText(frame,str(no_counting), (10, 180),cv2.FONT_HERSHEY_PLAIN, 4, color, 2)
            cv2.putText(frame,f'left_knee_angle :{str(angless1)}', (10, 60),cv2.FONT_HERSHEY_PLAIN, 1, color, 2)
            cv2.putText(frame,f'left_waist_angle :{str(angless2)}', (10, 75),cv2.FONT_HERSHEY_PLAIN, 1, color, 2)
            cv2.putText(frame,str(count), (10, 130),cv2.FONT_HERSHEY_PLAIN, 4, color1, 2)
            cv2.putText(frame,str(real_count), (10, 230),cv2.FONT_HERSHEY_PLAIN, 4, color2, 2)


        cv2.imshow('Pose Classification', frame)
        # Wait until a key is pressed.
        # Retreive the ASCII code of the key pressed
        k = cv2.waitKey(1) & 0xFF

        # Check if 'ESC' is pressed.
        if(k == 27):

            # Break the loop.
            break

# Release the VideoCapture object and close the windows.
    camera_video.release()
    cv2.destroyAllWindows()
# Create new thread to display video frame
t = threading.Thread(target=display_frame)
t.start()

# Main thread waits for speech to finish before continuing
while True:
    # Wait for event flag to be set
    event.wait()
    # Reset event flag to indicate speech is finished
    event.clear()
     


['perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'sit down more']
시각화 에서의  sit down more
스피크 함수에서의 sit down more
['perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'sit down more', 'sit down more', 'wake up more']
시각화 에서의  wake up more
스피크 함수에서의 wake up more


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\dwc06\anaconda3\envs\pose\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\dwc06\anaconda3\envs\pose\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\dwc06\AppData\Local\Temp\ipykernel_5556\1835363778.py", line 37, in speak_label
    engine.runAndWait()
  File "C:\Users\dwc06\anaconda3\envs\pose\lib\site-packages\pyttsx3\engine.py", line 177, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started



['perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'sit down more', 'sit down more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'perfect']
실행 순서에서 나오는 stack perfect
['perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'sit down more', 'sit down more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'wake up more']
시각화 에서의  wake up more
스피크 함수에서의 wake up more
['perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'pe

Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\dwc06\anaconda3\envs\pose\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\dwc06\anaconda3\envs\pose\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\dwc06\AppData\Local\Temp\ipykernel_5556\1835363778.py", line 37, in speak_label
    engine.runAndWait()
  File "C:\Users\dwc06\anaconda3\envs\pose\lib\site-packages\pyttsx3\engine.py", line 177, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started



실행 순서에서 나오는 stack straighten your back
['perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'sit down more', 'sit down more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'perfect']
['perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect'

...

실행 순서에서 나오는 stack sit down more
['perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'sit down more', 'sit down more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfe

실행 순서에서 나오는 stack sit down more
['perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'sit down more', 'sit down more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfe

['perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'sit down more', 'sit down more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perf

Exception in thread Thread-71:
Traceback (most recent call last):
  File "C:\Users\dwc06\anaconda3\envs\pose\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\dwc06\anaconda3\envs\pose\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\dwc06\AppData\Local\Temp\ipykernel_5556\1835363778.py", line 37, in speak_label
    engine.runAndWait()
  File "C:\Users\dwc06\anaconda3\envs\pose\lib\site-packages\pyttsx3\engine.py", line 177, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started



실행 순서에서 나오는 stack sit down more
['perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'sit down more', 'sit down more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'wake up more', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'straighten your back', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfect', 'perfe

In [ ]:
# 라이브러리 불러오기
import cv2
import threading
import time
import pyttsx3
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)
#tvideothread라는 스레스 생성. 스레드가 해야할 일들이 많기 때문에 클래스로 지정
class VideoThread(threading.Thread):
		# 객체가 생성될 때 호출된느 생섯아 함수. 객체를 초기화하고 클래스 멤버 변수를 설정하는데 사용됨.클래스의 인스턴스를 만들 때 호출 됨.
    def __init__(self, audio_event):
        super().__init__()
        self.mp_pose = mp.solutions.pose
        self.mp_drawing = mp.solutions.drawing_utils 
        self.pose_video = self.mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)
        self.camera_video= cv2.VideoCapture(0)
        self.camera_video.set(cv2.CAP_PROP_FPS, 20)#프레임 설정
        self.camera_video.set(3,1280)
        self.camera_video.set(4,960)
        self.event = threading.Event()
        self.color = (0, 0, 255)
        self.color1 = (128, 0, 0)
        self.color2 = (0,255,0)
        self.stack_label=[]
        self.stack_down_up=[]
        self.stack_in_action=[]
        self.no_counting=0
        self.real_count=0
        self.ar=np.empty((0,2),float)
        self.engine = pyttsx3.init()
        self.stack_label2=[]
        self.count=0
        self.new_label=[]
        self.audio_event=audio_event
        
        
		#스레드가 시작되면 직접적으로 호출되는 함수. 실행할 코드
    def run(self):
        while not self.event.is_set():
            while self.camera_video.isOpened():
                self.count=0
                ok, frame = self.camera_video.read()
                if not ok:
                    
                    # Continue to the next iteration to read the next frame and ignore the empty camera frame.
                    continue
    
                # Flip the frame horizontally for natural (selfie-view) visualization.
                frame = cv2.flip(frame, 1)
    
                # Get the width and height of the frame
                frame_height, frame_width, _ =  frame.shape
    
                # Resize the frame while keeping the aspect ratio.
                frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
    
                # Perform Pose landmark detection.
                frame, landmarks = detectPose(frame, self.pose_video, display=False)
    
                # Check if the landmarks are detected.
                if landmarks:
                # Perform the Pose Classification.)
                    frame, label = squat_classifyPose(landmarks, frame, display=False)
        
                   #앉고, 서고, 카운팅 
                    self.stack_label.append(label)
                    self.new_label = [key for key, group in itertools.groupby(self.stack_label) if key != 'Unknown Pose' ]
                    if len(self.new_label)>=2:
                        for i in range(len(self.new_label)-1):
                            if self.new_label[i:i+2] == ['sit Pose', 'stand Pose']:
                                self.count += 1
                    #노카운팅
                    angless1, angless2=squat_anglecheck(landmarks)
                    arr = np.array([[angless1,angless2]])
                    self.ar=np.append(self.ar,arr,axis=0)
                    ing=down_up(self.ar[:,0])
                    self.stack_down_up.append(ing)
                    in_actions=in_action(self.stack_down_up)
                    self.stack_in_action.append(in_actions)
                    label2=feedback_timing(self.stack_in_action,self.ar)

                    #시각화
                    if label2!=0:
                        cv2.putText(frame,str(label2), (10, 600),cv2.FONT_HERSHEY_PLAIN, 3, self.color, 2)
                        self.stack_label2.append(label2)

                    if len(self.stack_label2)>1:
                        if self.stack_label2[-1]!=self.stack_label2[-2] and self.stack_label2[-2]==self.stack_label2[-3]:
                            if self.stack_label2[-1]!='perfect':
                                self.event.set()
                                self.audio_event.set()
                                 #engine.runAndWait()
                    cv2.putText(frame,str(self.no_counting), (10, 180),cv2.FONT_HERSHEY_PLAIN, 4, self.color, 2)
                    cv2.putText(frame,f'left_knee_angle :{str(angless1)}', (10, 60),cv2.FONT_HERSHEY_PLAIN, 1, self.color, 2)
                    cv2.putText(frame,f'left_waist_angle :{str(angless2)}', (10, 75),cv2.FONT_HERSHEY_PLAIN, 1, self.color, 2)
                    cv2.putText(frame,str(self.count), (10, 130),cv2.FONT_HERSHEY_PLAIN, 4, self.color1, 2)
                    cv2.putText(frame,str(self.real_count), (10, 230),cv2.FONT_HERSHEY_PLAIN, 4, self.color2, 2)
                    self.count=0
            
                cv2.imshow('Pose Classification', frame)
    # Wait until a key is pressed.
    # Retreive the ASCII code of the key pressed
                k = cv2.waitKey(1) & 0xFF
    
    # Check if 'ESC' is pressed.
                if(k == 27):
                
        # Break the loop.
                    break

# Release the VideoCapture object and close the windows.
            self.camera_video.release()
            cv2.destroyAllWindows()


class AudioThread(threading.Thread):
    def __init__(self, audio_event):
        super().__init__()
        self.engine = pyttsx3.init()
        self.event = threading.Event()
        self.audio_event = audio_event

    def run(self):
        while self.event.is_set():
            # event.wait() 메서드를 사용하여 videoThread로부터 신호를 기다림
            self.event.wait()
            print(self.stack_label2[-1])
            self.engine.say(self.stack_label2[-1])
            self.engine.runAndWait()
            self.audio_event.clear()


if __name__ == "__main__":
    audio_event = threading.Event()
    videoThread = VideoThread(audio_event)
    audioThread = AudioThread(audio_event)

    videoThread.start()
    audioThread.start()

    videoThread.join()
    audioThread.join()